In [ ]:
pip install deepface
pip install opencv-python

In [ ]:
from deepface import DeepFace
import cv2

In [ ]:
# Setup video 
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


In [ ]:
# Identify which people are present in a specific frame 

def check_face(frame):
    
    # Compare all faces with those in the database 
    faces = DeepFace.find(frame, "database") 
    
    if not faces:
        return set()
    
    names = set() # Names of all people present 
    
    # Find the closest matching individual for each face 
    for face in faces:
        
        matches = face['identity']
        print(matches)
        counter = {}
        counter["Unknown"] = 1
        
        for i in range(len(matches)):
            parts = matches[i].split('\\')
            name = parts[1]
            counter[name] = counter.get(name, 0) + 1
        
        print(counter)
        names.add(max(counter.keys(), key = lambda x: counter[x]))
    
    return list(names)                           
            

In [ ]:
while True:
    ret, frame = cap.read()  # Capture photo 
    
    # Find all faces for this photo 
    if ret:
        try: 
            people = check_face(frame)
        except:
            people = ['No faces']
    
    # Select corresponding output for sentence 
    output = ""
    
    if len(people) == 0:
        output = 'Nobody'
    elif len(people) == 1:
        output = people[0]
    else:
        for i in range(len(people) - 1):
            output += people[i] + ', '
        output += "and " + people[-1]
    
    
    cv2.putText(frame, output + ' found!', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        
    cv2.imshow("video", frame)
        
    key = cv2.waitKey(1)
    if key == ord('q'):
        break 

cv2.destroyAllWindows()